In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from cleverhans.tf2.attacks import fast_gradient_method as FGSM

In [ ]:
df = pd.read_csv("Data/multi_data.csv")

Train, Test = train_test_split(df, test_size=0.33, random_state=69)

df_tensor_input_Train=tf.convert_to_tensor(Train.drop(['intrusion','Dos','normal','Probe','R2L','U2R','label'], axis=1))
df_tensor_output_Train=tf.convert_to_tensor(Train[['Dos','normal','Probe','R2L','U2R']]) 

df_tensor_input_Test=tf.convert_to_tensor(Test.drop(['intrusion','Dos','normal','Probe','R2L','U2R','label'], axis=1))
df_tensor_output_Test=tf.convert_to_tensor(Test[['Dos','normal','Probe','R2L','U2R']]) 

In [ ]:
mlp = tf.keras.Sequential()
mlp.add(tf.keras.layers.Dense(units=48, input_dim=df_tensor_input_Train.shape[1], activation="relu"))
mlp.add(tf.keras.layers.Dense(units=5, activation="softmax"))
mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
mlp.summary()

mlp.fit(df_tensor_input_Train, df_tensor_output_Train, epochs=100)

pred = mlp.predict(df_tensor_input_Test)

for j in range(0,pred.shape[1]):
  for i in range(0,pred.shape[0]):
    pred[i][j] = int(round(pred[i][j]))

pred_df = pd.DataFrame(pred,columns=Train[['Dos','normal','Probe','R2L','U2R']].columns)

print("Recall Score - ",recall_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8'),average='micro'))
print("F1 Score - ",f1_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8'),average='micro'))
print("Precision Score - ",precision_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8'),average='micro'))
print("Accuracy Score - ", accuracy_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8')))

In [ ]:
adv = FGSM.fast_gradient_method(mlp, df_tensor_input_Train, 0.2, np.inf)
adv_df = pd.DataFrame(adv.numpy())
adv_df.columns = Train.columns.drop(['intrusion','Dos','normal','Probe','R2L','U2R','label'])

mlp.fit(adv_df, df_tensor_output_Train, epochs=10)

pred = mlp.predict(df_tensor_input_Test)


In [ ]:
pred_df = pd.DataFrame(pred,columns=Train[['Dos','normal','Probe','R2L','U2R']].columns)


In [ ]:
pred_df.max(axis=1).describe()

In [ ]:
pred_df.iloc[[0]].max(axis=1)

In [ ]:
for i in range(0,pred_df.shape[0]):
    pred_df.where(pred_df.iloc[[i]]==pred_df.iloc[[i]].max(axis=1))

In [ ]:

for j in range(0,pred.shape[1]):
  for i in range(0,pred.shape[0]):
    pred[i][j] = int(round(pred[i][j]))

pred_df = pd.DataFrame(pred,columns=Train[['Dos','normal','Probe','R2L','U2R']].columns)

print("Recall Score - ",recall_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8'),average='micro'))
print("F1 Score - ",f1_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8'),average='micro'))
print("Precision Score - ",precision_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8'),average='micro'))
print("Accuracy Score - ", accuracy_score(Test[['Dos','normal','Probe','R2L','U2R']],pred_df.astype('uint8')))

In [ ]:
pred.shape

In [ ]:
true_state = []
pred_state = []

for i in range(0,pred.shape[0]):
    lab = ["Dos","normal","Probe","R2L","U2R"]
    for j in range(0,pred.shape[1]):
        if pred[i][j]==1:
            true_state.append(Test["label"])
            pred_state.append(lab[j])